## Get the data
csv headings: id, created_at, source, original_text, clean_text, favorite_count, retweet_count, hashtags, trend <br>
hashtags format: strings with comma separated hashtags

In [ ]:
#File paths
US_tweets_file = './Data/USTweets.csv'
UK_tweets_file = './Data/UKTweets.csv'

In [ ]:
import csv

tweets = []
hashtags = [] #list of lists of hashtags e.g. hashtags[0] = ["hashtag1", "hashtag2"]
hashtags_strings = [] #list of hashtags string e.g. hashtags[0] = ["hashtag1, hashtag2"]


UK data

In [ ]:
with open(UK_tweets_file) as data_file:
    data = csv.reader(data_file)
    next(data) #To skip the headings
    for row in data:
        tweets.append(row[4])
        hashtags.append(row[7].split(", "))
        hashtags_strings.append(row[7])

US data

In [ ]:
# with open(US_tweets_file) as data_file:
#     data = csv.reader(data_file)
#     next(data) #To skip the headings
#     for row in data:
#         tweets.append(row[4])
#         hashtags.append(row[7].split(", "))
#         hashtags_strings.append(row[7])

## Initialize the tokenizers
Will use a specialized tokenizer for the hashtags because we need to encode all the hashtags. It also does not matter if the encoding of the tweets match the encoding of the hashtags.

In [ ]:
from keras.preprocessing.text import Tokenizer

tweets_tokenizer = Tokenizer(oov_token="<OOV>")
tweets_tokenizer.fit_on_texts(tweets)
tweets_word_index = tweets_tokenizer.word_index
tweets_index_word = tweets_tokenizer.index_word

In [ ]:
hashtags_tokenizer = Tokenizer(oov_token="<OOV>")
hashtags_tokenizer.fit_on_texts(hashtags_strings)
hashtags_word_index = hashtags_tokenizer.word_index
hashtags_index_word = hashtags_tokenizer.index_word

In [ ]:
print(f'There are {len(tweets)} tweets, ')
print(f'the tweets contain {len(tweets_tokenizer.word_index)} different words.')
print(f'There are {len(hashtags_tokenizer.word_index)} different hashtags')
print('Here are the tokenized hashtags')
print(hashtags_word_index)

## Prepare the pre-trained embeddings

In [ ]:
import pickle
import numpy as np

num_tokens = len(tweets_word_index) + 1
embedding_dim = 300
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
with open('../Hashtag-recomender-embeddings/embeddings_index_object.pkl', 'rb') as embeddings_file:
    embeddings_index = pickle.load(embeddings_file)
    for word, i in tweets_word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            if embedding_vector.shape == (300,):
                embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1

In [35]:
print(f'hits:{hits}, misses: {misses}')

hits:8633, misses: 1734


## Create the sequences and pad them and one multi-hot encode the hashtags
Will use a binary vector to encode the hashtags to the model can categorize the tweets. e.g. hashtags[0] = [tag1, tag2], and tag1 has encoding of 1 and tag2 has encoding 2, then the binary vector wil be [0 1 1 0 0 ... no_of_different_hashtags]

In [ ]:
sequence_length = 10

from keras.utils import pad_sequences
tweets_sequences = tweets_tokenizer.texts_to_sequences(tweets)
hashtags_sequences = hashtags_tokenizer.texts_to_sequences(hashtags)
tweets_sequences_padded = pad_sequences(tweets_sequences, padding="post", maxlen=sequence_length)

In [ ]:
from keras.utils import to_categorical
import tensorflow as tf

no_of_different_hashtags = len(hashtags_word_index) + 1
no_of_hashtags = len(hashtags_sequences)

encoded_hashtags = np.zeros((no_of_hashtags, no_of_different_hashtags))

for i, hashtags_indices in enumerate(hashtags_sequences):
    encoded_hashtags[i][hashtags_indices] = 1


encoded_hashtags = np.array(encoded_hashtags)


## Split the data

In [ ]:
training_split = 0.8
training_tweets_count = int(0.8 * len(tweets_sequences_padded))

In [ ]:
train_data = tweets_sequences_padded[0:training_tweets_count]
train_labels = encoded_hashtags[0:training_tweets_count]
test_data = tweets_sequences_padded[training_tweets_count:]
test_labels = encoded_hashtags[training_tweets_count:]

print(f'we have {len(train_data)} tweets for training and {len(test_data)} for testing')

## Build the model

In [32]:
#hyperparameters
embedding_dimensions = 300
lstm_units = 128
dropout_value = 0.2
conv_filters = 64
conv_kernel_size = 5

In [33]:
from keras import initializers

no_of_tweets_words = len(tweets_word_index) + 1

hashtag_recommender_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(no_of_tweets_words, embedding_dimensions,
                               input_length=sequence_length,
                                 embeddings_initializer=initializers.Constant(embedding_matrix),
                                   trainable=False),
    # tf.keras.layers.Conv1D(conv_filters, conv_kernel_size, activation='relu'),
    tf.keras.layers.Dropout(dropout_value),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True)),
    # tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units)),
    tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Dense(no_of_different_hashtags, activation='softmax')
])

hashtag_recommender_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics = ['accuracy']
)

hashtag_recommender_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 10, 300)           3110400   
                                                                 
 dropout_4 (Dropout)         (None, 10, 300)           0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              439296    
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 2180)              560260    
                                                                 
Total params: 4,109,956
Trainable params: 999,556
Non-trainable params: 3,110,400
______________________________________

## Train the model

In [34]:
epochs = 30
hashtag_recommender_model.fit(train_data, train_labels, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/30
174/174 [==============================] - 7s 25ms/step - loss: 8.6174 - accuracy: 0.4107 - val_loss: 8.3941 - val_accuracy: 0.5583
Epoch 2/30
174/174 [==============================] - 4s 21ms/step - loss: 8.6253 - accuracy: 0.5642 - val_loss: 10.2885 - val_accuracy: 0.5950
Epoch 3/30
174/174 [==============================] - 4s 22ms/step - loss: 10.3207 - accuracy: 0.5730 - val_loss: 13.0169 - val_accuracy: 0.5899
Epoch 4/30
174/174 [==============================] - 4s 22ms/step - loss: 12.0543 - accuracy: 0.5937 - val_loss: 16.8882 - val_accuracy: 0.6115
Epoch 5/30
174/174 [==============================] - 4s 21ms/step - loss: 14.1731 - accuracy: 0.5967 - val_loss: 18.4011 - val_accuracy: 0.6000
Epoch 6/30
174/174 [==============================] - 4s 20ms/step - loss: 15.9646 - accuracy: 0.6021 - val_loss: 19.8425 - val_accuracy: 0.5950
Epoch 7/30
174/174 [==============================] - 3s 19ms/step - loss: 17.8673 - accuracy: 0.6052 - val_loss: 24.2497 - val_accur

## Get hashtags!!

In [ ]:
def predict(tweet, tweet_tokenizer, hashtag_tokenizer, pad_length, model):
    tweet_sequence = tweet_tokenizer.texts_to_sequences([tweet])[0]
    padded_tweet_sequence = pad_sequences([tweet_sequence], maxlen=pad_length, padding='post')
    prediction = (model.predict(padded_tweet_sequence))
    hashtag_indices = np.argsort(prediction, axis=-1)[0][-3:]
    return [hashtag_tokenizer.index_word[hashtag_index] for hashtag_index in hashtag_indices]


In [28]:
print(predict('great match', tweets_tokenizer, hashtags_tokenizer, 10, hashtag_recommender_model))

1/1 [==============================] - 1s 656ms/step
['barçaatleti', 'nufc', 'newtot']
